In [1]:
#load the modules
import os
import time
import numpy as np
from deepforest import main 
from deepforest import get_data
from deepforest import utilities
from deepforest import preprocess

/home/ubuntu/.conda/envs/drones4ducks/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Prepare training and validation data
trainimg_dir = r"/datadrive/Data/train/"
valimg_dir = r"/datadrive/Data/val/"

annotations_file = r"/datadrive/Data/train_filtered.csv"
validation_file = r"/datadrive/Data/val_filtered.csv"
#Find annotation path
#train_annotation_path = os.path.join(trainimg_dir,"train_example.csv")
#val_annotation_path = os.path.join(valimg_dir, "")


In [3]:
#ONLY IF WORKING WITH ORTHOMOSAICS: crop raster + save tiles in a newly created directory
#os.mkdir(os.getcwd(),'train_data_folder')
#crop_dir = os.path.join(os.getcwd(),'train_data_folder')
#train_annotations= preprocess.split_raster(path_to_raster=trainlabels,
#                                 annotations_file=annotation_path,
#                                 base_dir=crop_dir,
#                                 patch_size=400,
#                                 patch_overlap=0.05)

In [4]:
# #THIS was done in a separate script for this instance
# # Split image set into train / val / test. This is just a random subsampling by image; a spatial split is preferred where possible
# image_paths = train_annotations.filename.unique()
# #split 60/20/20 train/val/test
# valid_paths = np.random.choice(image_paths, int(len(image_paths)*0.4) )
# valid_annotations = train_annotations.loc[train_annotations.filename.isin(valid_paths)]
# train_annotations = train_annotations.loc[~train_annotations.filename.isin(valid_paths)]
# test_paths = np.random.choice(valid_paths, int(len(valid_paths)*0.5))
# test_annotations = valid_annotations.loc[valid_annotations.filename.isin(test_paths)]
# valid_annotations = valid_annotations.loc[~valid_annotations.filename.isin(test_paths)]

In [5]:
# #View output
# train_annotations.head()
# print("There are {} training bird annotations".format(train_annotations.shape[0]))
# #probably want to add other metrics-- class breakdown for instance
# print("There are {} test bird annotations".format(valid_annotations.shape[0]))

# #save to file and create the file dir
# annotations_file= os.path.join(img_dir,"train.csv")
# validation_file= os.path.join(img_dir,"valid.csv")
# #Write window annotations file without a header row, same location as the "base_dir" above.
# train_annotations.to_csv(annotations_file,index=False)
# valid_annotations.to_csv(validation_file,index=False)

In [6]:
#initial the model 
m = main.deepforest()

#change the corresponding config file
m.config['gpus'] = '-1' #move to GPU and use all the GPU resources
m.config["train"]["csv_file"] = annotations_file
m.config["train"]["root_dir"] = trainimg_dir
m.config["score_thresh"] = 0.4
m.config["train"]['epochs'] = 2
m.config["validation"]["csv_file"] = validation_file
m.config["validation"]["root_dir"] = valimg_dir

#create a pytorch lighting trainer used to training 
m.create_trainer()

#use Bird Detector
m.use_bird_release()

Reading config file: /home/ubuntu/.conda/envs/drones4ducks/lib/python3.10/site-packages/deepforest/data/deepforest_config.yml


/home/ubuntu/.conda/envs/drones4ducks/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/.conda/envs/drones4ducks/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/ubuntu/.conda/envs/drones4ducks/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus='-1')` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices='-1')` instead.
  ran

Model from BirdDetector Repo release https://github.com/weecology/BirdDetector/releases/tag/1.1 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/BirdDetector/releases/tag/1.1


In [7]:
start_time = time.time()
m.trainer.fit(m)
print(f"--- Training on GPU: {(time.time() - start_time):.2f} seconds ---")

ReferenceError: weakly-referenced object no longer exists

In [ ]:
#save the prediction result to a prediction folder
save_dir = os.path.join(os.getcwd(),'prediction_result')
try:
  os.mkdir(save_dir)
except FileExistsError:
  pass
results = m.evaluate(annotations_file, os.path.dirname(annotations_file), iou_threshold = 0.4, savedir= save_dir)

In [ ]:
##EVALUATION
results 

In [ ]:
results['box_precision']

In [ ]:
results["box_recall"]

In [ ]:
results["results"]

In [ ]:
results["class_recall"]